Define the brickcomps 

In [106]:
using DataFrames
using CSV
AIS_Data = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP85/projections_antarctic_RCP85_sneasybrick.csv") |> DataFrame

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,-0.0934139,-0.0582602,-0.106767,-0.188218,-0.0883429,-0.0881743,-0.0977073,-0.139816,0.0139191,-0.0693862,-0.146666,-0.124418,-0.0169269,-0.10725,-0.131933,-0.0393528,-0.113801,-0.0574834,-0.0823624,-0.146883,-0.10732,-0.0971754,-0.125281,-0.159976,-0.0496908,-0.128692,-0.141509,-0.0412395,-0.1063,-0.0644029,0.00859617,-0.132504,-0.133384,-0.105346,-0.113756,-0.126078,-0.0123916,-0.128283,-0.0386184,-0.0855744,-0.0818913,-0.0539375,-0.0161752,-0.166845,-0.0830161,-0.134765,-0.0856785,-0.168525,-0.061619,-0.0761899,-0.134013,-0.0854437,-0.0673604,-0.0930554,-0.0779066,-0.0185887,-0.0868476,-0.16168,-0.129047,-0.145243,-0.108583,-0.0612816,-0.15241,0.00523483,-0.120117,-0.102936,-0.15055,-0.0183506,-0.152137,-0.112692,-0.0978859,-0.178294,-0.0494852,-0.0790563,-0.131499,-0.0474008,-0.122018,-0.0771209,0.00520891,-0.0679831,-0.0995008,-0.106711,-0.056768,-0.117374,-0.0895482,-0.13346,-0.0527298,-0.143225,-0.0853682,-0.170037,-0.0902253,-0.049024,-0.103592,-0.111851,-0.0958981,-0.108939,-0.150283,-0.0861532,0.0401806,-0.122933,⋯
2,-0.132065,-0.111897,-0.0890768,-0.142199,-0.128172,-0.139149,-0.0538326,-0.131565,-0.0272952,-0.0710179,-0.10375,-0.121745,-0.0592255,-0.145903,-0.100606,-0.058975,-0.13178,-0.0677494,-0.0869661,-0.143396,-0.133004,-0.118263,-0.0776335,-0.127561,-0.0806523,-0.130101,-0.131354,-0.0543071,-0.115863,-0.0432105,-0.0410122,-0.142912,-0.137746,-0.122468,-0.0890547,-0.123426,-0.0580376,-0.150671,-0.0698963,-0.125207,-0.114518,-0.0659262,-0.0560155,-0.126997,-0.0789085,-0.123199,-0.128164,-0.124693,-0.0411151,-0.0571124,-0.129435,-0.12721,-0.0283234,-0.0809573,-0.0430068,-0.0513517,-0.121829,-0.124701,-0.117809,-0.112567,-0.130955,-0.0879537,-0.127155,-0.03638,-0.124662,-0.149595,-0.112082,-0.0588653,-0.127819,-0.112092,-0.0593039,-0.162312,-0.0993769,-0.0594125,-0.137942,-0.0342583,-0.122589,-0.113103,-0.0476559,-0.0786856,-0.0645,-0.0782243,-0.0735323,-0.131888,-0.0627343,-0.108662,-0.0736306,-0.134133,-0.0802848,-0.136622,-0.132303,-0.0238915,-0.129638,-0.134792,-0.101186,-0.106171,-0.135334,-0.0450632,-0.00941323,-0.12154,⋯
3,-0.132063,-0.109372,-0.0879083,-0.140583,-0.127467,-0.13929,-0.0535271,-0.130101,-0.0277738,-0.0705521,-0.10333,-0.12038,-0.0590263,-0.144199,-0.102198,-0.0591021,-0.130726,-0.0664827,-0.085434,-0.143643,-0.132437,-0.118968,-0.0774926,-0.126877,-0.0793884,-0.128223,-0.130317,-0.0541415,-0.115843,-0.0422542,-0.0410373,-0.141105,-0.138351,-0.122056,-0.0885937,-0.121992,-0.0580928,-0.149406,-0.0687752,-0.124022,-0.112847,-0.065105,-0.0552869,-0.126334,-0.078617,-0.122159,-0.126139,-0.122763,-0.0404296,-0.0573563,-0.128728,-0.127492,-0.028081,-0.0812386,-0.0427253,-0.0511894,-0.122698,-0.124468,-0.117224,-0.111189,-0.130407,-0.0889125,-0.127108,-0.0363624,-0.121878,-0.148227,-0.110999,-0.0582405,-0.127366,-0.11

In [107]:
GMSL_Data = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP85/projections_gmsl_RCP85_sneasybrick.csv") |> DataFrame

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,-0.154001,-0.152378,-0.13832,-0.190903,-0.0909671,-0.107157,-0.107471,-0.175071,0.00329118,-0.124439,-0.241963,-0.191096,-0.0374594,-0.107977,-0.171806,-0.0406856,-0.132156,-0.0386427,-0.0776097,-0.157261,-0.196005,-0.139731,-0.171433,-0.219722,-0.0861081,-0.126549,-0.170693,-0.0727316,-0.138534,-0.132613,-0.0442634,-0.167135,-0.135552,-0.13552,-0.15154,-0.195733,-0.0503788,-0.137933,-0.056919,-0.110344,-0.174851,-0.121954,-0.0261881,-0.190627,-0.102319,-0.186949,-0.0985081,-0.219043,-0.125521,-0.0765,-0.192236,-0.113795,-0.123018,-0.135911,-0.0999893,-0.0635513,-0.125339,-0.161951,-0.143,-0.198579,-0.128625,-0.0947574,-0.183687,-0.0602605,-0.148709,-0.17422,-0.190984,-0.0113189,-0.206109,-0.141493,-0.164335,-0.24565,-0.071948,-0.135471,-0.180145,-0.0694726,-0.15152,-0.152766,-0.0279172,-0.129846,-0.128634,-0.144433,-0.151635,-0.170574,-0.109084,-0.179444,-0.118101,-0.183775,-0.138527,-0.197735,-0.122748,-0.0633126,-0.105614,-0.158245,-0.135082,-0.161152,-0.198097,-0.0942303,-0.0169109,-0.17072,⋯
2,-0.173539,-0.126757,-0.167549,-0.144453,-0.138944,-0.122211,-0.0880347,-0.182372,-0.0798365,-0.0993328,-0.17468,-0.13596,-0.104102,-0.133068,-0.12258,-0.10387,-0.124668,-0.115195,-0.148217,-0.193955,-0.145567,-0.181728,-0.105204,-0.13488,-0.105452,-0.163376,-0.140655,-0.0947886,-0.131438,-0.0729415,-0.090269,-0.172844,-0.182573,-0.171943,-0.1445,-0.168406,-0.101705,-0.185707,-0.108433,-0.155026,-0.118469,-0.0990595,-0.0828548,-0.134144,-0.14257,-0.156891,-0.181147,-0.209676,-0.0102027,-0.0733447,-0.177543,-0.157737,-0.0881197,-0.124282,-0.0906836,-0.111802,-0.164702,-0.168881,-0.142494,-0.139035,-0.134444,-0.142102,-0.146924,-0.0903349,-0.17394,-0.189283,-0.147293,-0.0909113,-0.20083,-0.128393,-0.0951324,-0.161506,-0.159845,-0.0795949,-0.181114,-0.0650037,-0.170889,-0.151751,-0.0732168,-0.11555,-0.129708,-0.119998,-0.114972,-0.171302,-0.103899,-0.113348,-0.098487,-0.137117,-0.106738,-0.177524,-0.191751,-0.0954137,-0.179396,-0.148827,-0.137466,-0.180131,-0.207205,-0.0676596,-0.0527101,-0.136927,⋯
3,-0.184835,-0.155199,-0.130034,-0.140501,-0.148099,-0.151268,-0.105485,-0.172311,-0.0764371,-0.113145,-0.199371,-0.187471,-0.0547421,-0.145373,-0.101868,-0.077927,-0.134934,-0.0892489,-0.109227,-0.160074,-0.190971,-0.145237,-0.117072,-0.151532,-0.121563,-0.144616,-0.154828,-0.0738653,-0.136788,-0.0541951,-0.0951703,-0.225584,-0.136964,-0.19739,-0.114792,-0.153573,-0.0889946,-0.16675,-0.135063,-0.152177,-0.162135,-0.0911389,-0.123353,-0.143582,-0.103151,-0.191191,-0.138561,-0.212993,-0.0740284,-0.0714797,-0.187682,-0.126082,-0.0842235,-0.106103,-0.0974829,-0.121622,-0.171157,-0.209023,-0.166238,-0.131618,-0.18495,-0.13683,-0.164666,-0.0708538,-0.189106,-0.136018,-0.185622,-0.109764,-0.206188,-0.159123,-0.1073,-0.12601,-0.146892,-0.0694727,-0.132867,-0.0538786,-0

In [108]:
LWS_Data = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP85/projections_landwater_storage_sl_RCP85_sneasybrick.csv") |> DataFrame

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,0.000496146,0.000106584,-2.23019e-6,-8.11728e-5,0.000388606,4.99594e-6,0.000335458,0.000158677,0.000545394,0.000376077,0.000446673,0.000249793,0.000139337,0.000129145,0.000151125,0.000367747,0.000392705,0.000496531,0.000180125,0.000498467,0.000481855,0.000106899,0.000166341,0.000269996,0.000341926,0.000324773,0.000459336,0.000481268,0.000167592,0.000340446,0.000156154,0.000244799,0.000229542,0.000592194,0.00046041,0.000681589,0.000225671,0.000349479,0.00026822,9.67981e-6,0.000162743,0.000332611,-0.000148713,0.000306514,0.000245782,0.000651395,9.48799e-5,0.000530831,0.000368492,0.000277199,0.00014758,0.000212989,0.000276028,0.000325665,7.30545e-5,0.000199164,0.000265436,0.000199263,0.000575955,9.61998e-5,0.00020116,0.000227126,0.000154663,0.000176058,0.000426106,0.000235844,0.000604435,0.000433126,0.000350387,0.00047649,0.000148359,0.000488122,0.000316926,0.000445998,3.33093e-5,0.000417527,0.00010327,-1.28879e-5,0.000362553,0.000769173,0.000278842,0.000402835,0.000132481,0.000679745,0.000189768,0.000268555,0.000221443,0.000393161,0.000383309,0.000197742,8.34066e-5,0.000572427,1.66514e-5,-9.35755e-5,0.000305661,0.000223844,0.000409686,7.12057e-5,4.1231e-5,0.000348794,⋯
2,0.000236644,0.000201135,8.73142e-5,5.06856e-5,0.000342735,0.000111497,8.90539e-5,9.12605e-5,5.83906e-5,0.000395437,0.000135488,0.00026372,0.000794582,0.000402274,0.000110165,-1.70105e-5,0.000221592,3.69131e-5,0.000344426,0.000347161,0.000345692,0.000256035,0.000106915,-6.56667e-5,0.000516117,0.000261003,0.000194701,0.000260733,0.000325385,0.000234842,0.000669414,0.000262874,0.000329284,1.77515e-5,-7.51398e-5,0.000235367,9.53713e-5,0.000349244,0.000341531,0.000472306,0.000341824,0.000304502,0.000193358,0.000222471,0.000346346,0.000253409,0.0001715,0.000176981,8.15138e-5,0.000426117,-0.000112139,-1.05944e-5,0.000431404,0.000174646,0.000515301,0.000150642,0.000173481,0.00054971,0.000187974,8.36516e-5,0.000381792,0.000390904,0.000301054,-3.13723e-5,0.000443288,0.000310987,0.000275798,0.000357797,2.81819e-5,0.000276458,0.000241084,0.000356672,6.34461e-5,0.000357061,0.000403537,0.000273644,0.000503072,0.000241888,0.000300396,0.000338892,0.00035671,0.000207141,0.000423824,8.96928e-5,0.000125086,0.000628234,0.000501144,0.000407895,0.000388401,0.000272455,0.000570292,0.000329922,0.000223882,0.000314892,0.000380724,0.000453283,0.000470209,0.000244469,0.000413996,0.000594385,⋯
3,0.000363723,0.000431675,6.96856e-5,-2.83181e-5,0.000475926,0.000469621,1.68516e-5,6.07294e-5,-0.000134763,-0.000196604,0.000214059,0.000390558,0.000230287,0.000409781,0.00052769,0.000443958,0.000361483,0.000319456,0.000209093,9.78655e-5,0.000379009,0.000248308,0.000489812,-2.24798e-5,0.000379749,0.000485994,0.000298025,0.000194603,0.000183854,0.00047571,0.000181257,0.000399586,0.00043407,0.000306498,0.000524235,0.00031

In [109]:
TE_Data = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP85/projections_thermal_RCP85_sneasybrick.csv") |> DataFrame

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,0.0227005,0.0131938,0.0332008,-0.0035008,0.0230374,-0.0117678,0.0187602,0.00887515,-0.0178687,0.00101321,0.00934179,0.0435676,-0.037683,-0.0201504,0.0377008,0.0240619,0.0457134,0.0141789,0.00708843,0.00507343,-0.0375809,-0.019016,-0.00302912,0.0231058,-0.0481778,0.00478629,-0.0150074,-0.0409779,0.0295779,0.0421613,0.0167979,-0.0234762,-0.0408617,0.0256427,0.023044,-0.0306744,-0.0136032,-0.025845,0.038946,-0.0206836,0.006431,8.23721e-5,0.00315497,0.0326341,-0.015068,0.00210088,-0.0179501,-0.0309719,-0.0331044,0.0391193,-0.024225,0.0245688,0.0160355,0.0253622,0.0248517,0.00723821,0.037172,-0.00530509,-0.0123202,0.0319672,0.0340078,0.01255,-0.0409602,-0.00425895,0.00300298,0.0187061,-0.0352274,-0.0438027,0.034974,-0.00752335,-0.0165473,-0.0365918,-0.022084,0.0174211,-0.0126938,-0.0180633,0.00287071,0.0294764,-0.0206506,0.0345616,0.0119561,0.0373978,-0.0362339,0.0427932,-0.030829,-0.0122564,-0.0319476,0.0187005,-0.031449,-0.0234198,0.0323051,-0.0177504,0.00668436,-0.00375302,-0.0088625,-0.000558067,0.00953175,-0.0373185,-0.0169966,-0.0025531,⋯
2,0.023083,0.0137543,0.0335027,-0.00315398,0.0234456,-0.0114322,0.0191611,0.00932629,-0.0175512,0.00131142,0.00980213,0.0439187,-0.0374044,-0.0198834,0.0379463,0.0243539,0.0460208,0.0145409,0.00742568,0.00533764,-0.0372133,-0.0186214,-0.00261832,0.0234545,-0.0477161,0.00511625,-0.0146961,-0.0405471,0.0298922,0.0425512,0.0171464,-0.0231544,-0.0405544,0.0260203,0.0234162,-0.0302972,-0.0132299,-0.0254112,0.0394291,-0.0203294,0.00685103,0.000441189,0.00356005,0.0330045,-0.0147334,0.00251187,-0.0175946,-0.0306697,-0.0327935,0.0394595,-0.0238663,0.0249663,0.0163914,0.0257694,0.0252363,0.00766679,0.0375157,-0.00488788,-0.0119766,0.032436,0.0344218,0.0129108,-0.0405886,-0.00395839,0.0033872,0.0191016,-0.0348619,-0.0435141,0.0353176,-0.00708572,-0.016135,-0.0363877,-0.0216406,0.0177312,-0.0122432,-0.0177806,0.00332805,0.0299442,-0.0201842,0.0350377,0.0122539,0.0377572,-0.0358743,0.0431959,-0.0304222,-0.0118691,-0.0316111,0.0190285,-0.0309167,-0.0229989,0.0326491,-0.0173948,0.00698576,-0.00345834,-0.00840549,-0.000263582,0.00997798,-0.0368882,-0.0166713,-0.00206285,⋯
3,0.0234567,0.0142628,0.0338091,-0.00280312,0.0238427,-0.0110971,0.0195265,0.00974732,-0.0172395,0.0015979,0.0102496,0.0442643,-0.0371312,-0.0196099,0.0381935,0.0246387,0.0463289,0.014892,0.00774581,0.00560366,-0.0368532,-0.0182473,-0.00221233,0.0237983,-0.0472941,0.00543788,-0.0143867,-0.0401411,0.030197,0.0429345,0.0174872,-0.0228439,-0.04025,0.0263799,0.0237835,-0.0299365,-0.0128746,-0.0249989,0.0398976,-0.0199873,0.00725209,0.000792663,0.00393402,0.0333644,-0.0144165,0.00290799,-0.0172407,-0.0303684,-0.0324923,0.0397916,-0.0235188,0.0253425,0.0167368,0.026148,0.025613,0.00807798,0.0378538,-0.0044873,-0.0116352,0.0328782,0.034811,0.0132626,-0.0402324,-0.00

In [110]:
GIS_Data = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP85/projections_greenland_RCP85_sneasybrick.csv") |> DataFrame

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,0.00146739,0.00853835,0.00258476,0.010421,0.00885618,0.00660128,0.00441786,0.0116467,0.00230798,0.00314623,0.00107353,0.00454809,0.00395916,0.00828802,0.00102872,0.00607462,0.0123293,0.00200293,-0.000373429,0.0081154,0.00433152,0.00585527,0.00157972,0.00604341,0.00418039,0.00155648,0.000305297,0.0022049,0.0177199,0.00320351,0.00167891,0.0027535,0.00222131,-0.00236151,0.00240005,-0.00333395,0.00261588,0.0145081,0.00125221,-1.79119e-5,-0.00122049,0.0112123,0.00612599,0.0163015,0.00308478,0.00299995,0.0211264,-0.00186495,0.00558115,0.00185146,0.00819399,0.00239397,0.0128952,0.00742434,0.00436949,0.00472859,0.00260314,4.35477e-5,-0.000611996,0.00240492,0.00459771,0.0105357,0.00563117,0.00683609,0.00123027,0.0111062,0.000936057,0.000305992,-0.000133864,0.00604006,0.0169004,0.0156866,0.00121152,0.00466102,0.0174776,0.00843475,5.32431e-5,0.00399287,0.00491086,0.00297758,0.00383671,-0.000254281,0.00505133,0.000928149,0.00993043,0.00657291,0.00662325,0.00263779,0.00366691,0.00370643,0.00562003,0.0200251,-0.000101986,0.0101682,0.0110957,-0.00201637,0.00300404,0.0049713,0.00270566,0.00688307,⋯
2,0.00199922,0.00884697,0.00203638,0.0111168,0.00820163,0.00663427,0.00427115,0.0103916,0.00221641,0.00323997,0.00132705,0.00412123,0.00404438,0.00822755,0.00129453,0.00613417,0.0128254,0.00186421,-0.000567052,0.00798165,0.00410033,0.00572479,0.00139002,0.00598296,0.00410976,0.00164418,0.000880312,0.00238004,0.0172847,0.0031114,0.00129889,0.00328089,0.00228908,-0.00287559,0.00179273,-0.00323552,0.00239848,0.0141912,0.00110892,0.000105295,-0.00127195,0.0110792,0.00611929,0.0163519,0.00363103,0.00281496,0.0216464,-0.00181275,0.0059436,0.00174919,0.00826218,0.00188502,0.012197,0.00673681,0.0035717,0.00498412,0.00295828,0.00083948,-0.00118104,0.0022989,0.00488104,0.0104147,0.00517927,0.00660799,0.000896349,0.0113782,0.00109277,0.000243129,-0.000109976,0.00604479,0.0168717,0.0155607,0.00162657,0.00473393,0.0171344,0.0082689,-0.000271963,0.00358598,0.00561908,0.00304686,0.00369187,0.000268596,0.00534659,0.0011739,0.00970591,0.00579591,0.00568338,0.00235103,0.00405041,0.00306185,0.00500292,0.0192919,-0.000557146,0.00892139,0.0109525,-0.00198942,0.00318061,0.00518543,0.00276378,0.00683128,⋯
3,0.0015515,0.00854035,0.00219757,0.010983,0.00835263,0.00639801,0.00431622,0.0105821,0.00204745,0.00314797,0.00125098,0.00450711,0.00410398,0.00832654,0.00120546,0.00640863,0.0122752,0.00234503,-0.000280083,0.00814642,0.00451966,0.00554475,0.00115814,0.00559309,0.00401828,0.0015946,0.000701107,0.00220247,0.0175937,0.00336133,0.00140858,0.00306967,0.00180782,-0.00259551,0.0017489,-0.00288913,0.00236417,0.013781,0.000726371,0.000422518,-9.9711e-5,0.0109828,0.00588314,0.0161922,0.00372305,0.00319048,0.0213956,-0.000849552,0.00602726,0.00190697,0.0076895,0.00240642,0.0128214,0.00672685,0.00311

In [111]:
GSIC_Data = CSV.File("/Users/ce3304/Downloads/projections_csv/RCP85/projections_glaciers_RCP85_sneasybrick.csv") |> DataFrame

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,-0.0300002,-0.0329503,-0.0287041,-0.0262065,-0.0254624,-0.0283992,-0.0332566,-0.0251137,-0.029877,-0.023214,-0.0311617,-0.027285,-0.0276051,-0.0263734,-0.0283867,-0.0273,-0.0302329,-0.031021,-0.0278943,-0.0345783,-0.0296118,-0.0290711,-0.0309096,-0.0317293,-0.0302478,-0.0277554,-0.0274059,-0.0379479,-0.0262448,-0.0293448,-0.0271107,-0.0295052,-0.0258021,-0.03083,-0.0288637,-0.0256869,-0.0262886,-0.0275305,-0.0296805,-0.0309617,-0.031532,-0.0299552,-0.0274167,-0.0313373,-0.0257661,-0.0272593,-0.0227471,-0.0328903,-0.0308485,-0.0297159,-0.0298608,-0.0285653,-0.0296757,-0.0285734,-0.029044,-0.0274977,-0.0319212,-0.0331464,-0.0267038,-0.0329646,-0.0305329,-0.0324013,-0.0327705,-0.0304005,-0.0270346,-0.0314867,-0.0285076,-0.0272829,-0.0311543,-0.0257663,-0.0262536,-0.0319194,-0.0278219,-0.0257006,-0.0304607,-0.0301918,-0.029602,-0.0274652,-0.0342069,-0.0267196,-0.0284968,-0.0284647,-0.0314638,-0.0316072,-0.0266754,-0.0248257,-0.0254983,-0.0315314,-0.0247968,-0.0306461,-0.0279329,-0.0288882,-0.0316559,-0.0292847,-0.0335577,-0.0226867,-0.0289671,-0.0281669,-0.0299291,-0.0280774,⋯
2,-0.0280272,-0.0347702,-0.0288519,-0.0262484,-0.0242614,-0.0236134,-0.0325597,-0.0310037,-0.0294297,-0.0274093,-0.029632,-0.0294614,-0.0275655,-0.0226096,-0.0279023,-0.0243563,-0.0282396,-0.0299835,-0.0272195,-0.0355436,-0.0275919,-0.0268202,-0.0268584,-0.0277404,-0.030947,-0.0271293,-0.0260012,-0.0362595,-0.0326536,-0.0282171,-0.027726,-0.0280428,-0.0230785,-0.0295967,-0.0312107,-0.0372586,-0.0281732,-0.0270612,-0.027679,-0.0286576,-0.0284738,-0.0336585,-0.0259738,-0.0298216,-0.0288781,-0.0305072,-0.030364,-0.0350852,-0.0287307,-0.0335412,-0.0299067,-0.0292341,-0.0324808,-0.0251844,-0.0340423,-0.0298709,-0.0256389,-0.0282382,-0.0293881,-0.0342022,-0.0300753,-0.0246272,-0.0340064,-0.0298,-0.0261478,-0.0283613,-0.027074,-0.027729,-0.0304791,-0.0274166,-0.0335916,-0.0301811,-0.0262331,-0.030012,-0.0282864,-0.0283549,-0.0260019,-0.0317382,-0.0288355,-0.0268526,-0.0243628,-0.0271415,-0.0292593,-0.0263767,-0.0298614,-0.0273337,-0.0294394,-0.0313191,-0.0286459,-0.0299682,-0.0257248,-0.0312499,-0.0333065,-0.0271157,-0.0323419,-0.0237445,-0.0283677,-0.0277787,-0.027253,-0.0268315,⋯
3,-0.0239259,-0.0321869,-0.029774,-0.0291132,-0.0262658,-0.0292183,-0.0318194,-0.0261626,-0.0277782,-0.0324372,-0.0293852,-0.0274458,-0.0313926,-0.0217731,-0.0284998,-0.027683,-0.0273574,-0.0276523,-0.0307877,-0.0343145,-0.0270482,-0.0273528,-0.0296263,-0.027579,-0.0331258,-0.0284088,-0.0226081,-0.0371319,-0.0263787,-0.0251908,-0.0298016,-0.0264009,-0.0257717,-0.031334,-0.0240252,-0.02915,-0.0265048,-0.0221675,-0.0293432,-0.0280026,-0.0283656,-0.0304791,-0.0293124,-0.0269049,-0.0277846,-0.027979,-0.0264208,-0.0323649,-0.0298713,-0.0282701,-0.0272551,-0.0282934,-0.0261862,-0.0293711,-0.0310269,-0.0300604,-0.0

Construct Brick components

In [124]:
Sim=10000
GMSL=GMSL_Data[:,1:Sim];
LWS=LWS_Data[:,1:Sim];
GIS= GIS_Data[:,1:Sim];
GSIC=GSIC_Data[:,1:Sim];
TE=TE_Data[:,1:Sim];
AIS=AIS_Data[:,1:Sim];

In [113]:
Y=collect(1850:2300);

btime=Y;



In [125]:
brickcomps=Tuple([btime,AIS,GSIC,GIS,TE,LWS,GMSL]);

Run Brick downscale function

In [115]:
function get_fingerprints()

    fp_dir = joinpath(@__DIR__, "..", "data")
    isdir(fp_dir) || mkpath(fp_dir)
    fp_file = joinpath(fp_dir, "FINGERPRINTS_SLANGEN_Bakker.nc")
    if !isfile(fp_file)
        url = "https://github.com/scrim-network/BRICK/raw/master/fingerprints/FINGERPRINTS_SLANGEN_Bakker.nc"
        download(url, fp_file)
    end

    fplat = ncread(fp_file,"lat")
    fplon = ncread(fp_file,"lon")
    fpAIS = ncread(fp_file,"AIS")
    fpGSIC = ncread(fp_file,"GLAC")
    fpGIS = ncread(fp_file,"GIS")
    ncclose()

    return fplat,fplon,fpAIS,fpGSIC,fpGIS
end

get_fingerprints (generic function with 1 method)

In [116]:
function subtractor(minval,maxval)
    function y(point,n)
        if point - n < minval
            return min(maxval,point - n + maxval)
        else
            return point - n
        end
    end
end

subtractor (generic function with 1 method)

In [117]:
function adder(maxval)
    function y(point,n)
        if point + n > maxval
            return point + n - maxval
        else
            return point + n
        end
    end
end

adder (generic function with 1 method)

In [118]:
lon_subtractor = subtractor(1,360)
lon_adder = adder(360)
lat_adder = adder(180)
lat_subtractor = subtractor(1,180)

(::var"#y#209"{Int64, Int64}) (generic function with 1 method)

In [119]:
using Statistics
using NetCDF
function downscale_brick(brickcomps, lonlat, ensInds, ystart=2010, yend=2100, tstep=10)
    # To do - check with vectors of lat, lon
    (fplat,fplon,fpAIS,fpGSIC,fpGIS) = get_fingerprints()
    (btime,AIS,GSIC,GIS,TE,LWS,GMSL) = brickcomps

    # Select indices of time of interest, with respect to timestep
    tinds = findall(x -> x .>= ystart && x .<=yend, btime)
    years = collect(ystart:yend)
    yinds = findall(x -> x % tstep==0, years)
    # Need to normalize LSL relative to 2000
    inorm = findall(x -> x==2000, btime)

    tdim=length(btime)

    if length(years)==length(tinds)
        tinds = tinds[yinds]
    else
        println("Error: years outside of bounds")
        return nothing
    end

    num_ens = length(ensInds)
    
    # Output matrix: ens x time x segment
    lsl_out = zeros(num_ens, length(tinds), length(lonlat))

    # Trim component vectors to timesteps and ensembles. Assume interval is 1 year
    if tdim==size(AIS)[1] # check that time dimension is 1
        # for normalizing
        AIS_norm = AIS[inorm,ensInds]
        GSIC_norm = GSIC[inorm,ensInds]
        GIS_norm = GIS[inorm,ensInds]
        TE_norm = TE[inorm,ensInds]
        LWS_norm = LWS[inorm,ensInds]
        GMSL_norm = GMSL[inorm,ensInds]
        # actual projections
        AIS = AIS[tinds,ensInds]
        GSIC = GSIC[tinds,ensInds]
        GIS = GIS[tinds,ensInds]
        TE = TE[tinds,ensInds]
        LWS = LWS[tinds,ensInds]
        GMSL = GMSL[tinds,ensInds]
    else
        println("Error: time dimension is not 1 for brick components")
        return nothing
    end

  

    for f in 1:length(lonlat) # Loop through lonlat tuples

        lon = lonlat[f][1]
        lat = lonlat[f][2]
        # Convert Longitude to degrees East
        # CIAM Lat is already in (-90,90) by default
        if lon <0
            lon = lon + 360
        end

        # Find fingerprint degrees nearest to lat,lon
        ilat = findall(isequal(minimum(abs.(fplat.-lat))),abs.(fplat.-lat))
        ilon = findall(isequal(minimum(abs.(fplon.-lon))),abs.(fplon.-lon))


        # Take average of closest lat/lon values
        fpAIS_flat = collect(skipmissing(Iterators.flatten(fpAIS[ilon,ilat])))
        fpGSIC_flat = collect(skipmissing(Iterators.flatten(fpGSIC[ilon,ilat])))
        fpGIS_flat = collect(skipmissing(Iterators.flatten(fpGIS[ilon,ilat])))

        fpAIS_loc = mean(fpAIS_flat[isnan.(fpAIS_flat).==false],dims=1)[1]
        fpGSIC_loc = mean(fpGSIC_flat[isnan.(fpGSIC_flat).==false],dims=1)[1]
        fpGIS_loc = mean(fpGIS_flat[isnan.(fpGIS_flat).==false],dims=1)[1]
        fpTE_loc = 1.0
        fpLWS_loc=1.0

        # Keep searching nearby lat/lon values if fingerprint value is NaN unless limit is hit
        inc =1

        while isnan(fpAIS_loc) || isnan(fpGIS_loc) || isnan(fpGSIC_loc) && inc<5

            newlonStart = lon_subtractor.(fplon[ilon],inc)[1]
            newlatStart = lat_subtractor.(fplat[ilat],inc)[1]
            newlonEnd = lon_adder.(fplon[ilon],inc)[1]
            newlatEnd = lat_adder.(fplat[ilat],inc)[1]

            latInd1 = minimum(findall(isequal(minimum(abs.(fplat.-newlatStart))),abs.(fplat.-newlatStart)))
            #minimum(findall(x-> x in newlatStart,fplat))
            latInd2 = maximum(findall(isequal(minimum(abs.(fplat.-newlatEnd))),abs.(fplat.-newlatEnd)))
            #maximum(findall(x -> x in newlatEnd,fplat))

            lonInd1 = minimum(findall(isequal(minimum(abs.(fplon.-newlonStart))),abs.(fplon.-newlonStart)))
            #minimum(findall(x-> x in newlonStart,fplon))
            lonInd2 = maximum(findall(isequal(minimum(abs.(fplon.-newlonEnd))),abs.(fplon.-newlonEnd)))
            #maximum(findall(x -> x in newlonEnd,fplon))

            if latInd2 < latInd1
                latInds=[latInd1; 1:latInd2]
            else
                latInds=latInd1:latInd2
            end

            if lonInd2 < lonInd1
                lonInds=[lonInd1; 1:lonInd2]
            else
                lonInds = lonInd1:lonInd2
            end

            fpAIS_flat = collect(skipmissing(Iterators.flatten(fpAIS[lonInds,latInds])))
            fpGSIC_flat = collect(skipmissing(Iterators.flatten(fpGSIC[lonInds,latInds])))
            fpGIS_flat = collect(skipmissing(Iterators.flatten(fpGIS[lonInds,latInds])))

            fpAIS_loc = mean(fpAIS_flat[isnan.(fpAIS_flat).==false],dims=1)[1]
            fpGSIC_loc = mean(fpGSIC_flat[isnan.(fpGSIC_flat).==false],dims=1)[1]
            fpGIS_loc = mean(fpGIS_flat[isnan.(fpGIS_flat).==false],dims=1)[1]

            inc = inc + 1

        end

        # If still NaN, throw an error
        if isnan(fpAIS_loc) || isnan(fpGIS_loc) || isnan(fpGSIC_loc)
            println("Error: no fingerprints found for ($(lon),$(lat))")
            return nothing
        end
       # Multiply fingerprints by BRICK ensemble members
       if ndims(AIS) > 1
        for n in 1:size(AIS)[2] # loop through ensemble members
            lsl_out[n, :, f] = fpGIS_loc * GIS[:,n] + fpAIS_loc * AIS[:,n] + fpGSIC_loc * GSIC[:,n] +
                               fpTE_loc * TE[:,n] + fpLWS_loc * LWS[:,n]
            # CIAM - LSL should be sea-level change relative to year 2000
            lsl_norm = fpGIS_loc * GIS_norm[1,n] + fpAIS_loc * AIS_norm[1,n] + fpGSIC_loc * GSIC_norm[1,n] +
                       fpTE_loc * TE_norm[1,n] + fpLWS_loc * LWS_norm[1,n]
            lsl_out[n, :, f] = lsl_out[n, :, f] .- lsl_norm
            
        end
    else
            lsl_out[1, :, f] = fpGIS_loc * GIS[:] + fpAIS_loc * AIS[:] + fpGSIC_loc * GSIC[:] +
                                fpTE_loc * TE[:] + fpLWS_loc * LWS[:]
            # CIAM - LSL should be sea-level change relative to year 2000
            lsl_norm = fpGIS_loc * GIS_norm + fpAIS_loc * AIS_norm + fpGSIC_loc * GSIC_norm +
                                fpTE_loc * TE_norm + fpLWS_loc * LWS_norm
            lsl_out[1, :, f] = lsl_out[1, :, f] .- lsl_norm
    end

    end # End lonlat tuple

    return lsl_out,GMSL
end

downscale_brick (generic function with 4 methods)

In [126]:
ensInds=collect(1:10000);
lonlat= ([-104.3385,19.1138],[-90.0715,29.9511]);
ystart=2010;
yend=2200;
tstep=10;

lslrNew,H=downscale_brick(brickcomps, lonlat, ensInds, ystart, yend, tstep)

([0.02676421779527241 0.06812314170115447 … 6.361669572580161 6.862617183909249; 0.024081657354711766 0.06299086519819136 … 3.673713132631384 3.99032120250646; … ; 0.032181918599016354 0.07211076206309919 … 5.156145274576946 5.615677439153576; 0.030560387249516846 0.06991447462590339 … 6.44974410335111 6.9719984662454415]

[0.025283754282903553 0.06390356159746934 … 5.682386744769553 6.1214555837968625; 0.02304732387304978 0.059257821539559385 … 3.3212798100616925 3.60458242479375; … ; 0.030655010921650183 0.06810956394438224 … 4.535057511755059 4.9348544856717895; 0.029054709859027222 0.06582189420482644 … 5.6864725459669945 6.140884458605818], 20×10000 DataFrame
 Row │ x1         x2         x3         x4         x5         x6         x7    ⋯
     │ Float64    Float64    Float64    Float64    Float64    Float64    Float ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ 0.0835504  0.0814309  0.0773876  0.0827359  0.0704417  0.0788344  0.066 ⋯
   

In [123]:
using MimiCIAM
using Mimi
M=zeros(1,10);
for i in 1:10
    new_lslr=zeros(20,2);
    for j in 1:2
        for k in 1:20
        new_lslr[k,j]=lslrNew[i,k,j];
        end 
     end 
    M[i] = MimiCIAM.get_model(initfile ="../ColimaUSA3_init.csv")
    update_param!(M[i], :slrcost, :lslr, )
    run(M[i])
end

MethodError: MethodError: Cannot `convert` an object of type Model to an object of type Float64
Closest candidates are:
  convert(::Type{T}, !Matched::DoubleFloats.DoubleFloat{DoubleFloats.DoubleFloat{T}}) where T<:Union{Float16, Float32, Float64} at /Users/ce3304/.julia/packages/DoubleFloats/6UuOC/src/type/convert.jl:37
  convert(::Type{T}, !Matched::Mimi.ScalarModelParameter{T}) where T at /Users/ce3304/.julia/packages/Mimi/OClro/src/core/types/params.jl:46
  convert(::Type{T}, !Matched::DualNumbers.Dual) where T<:Union{Real, Complex} at /Users/ce3304/.julia/packages/DualNumbers/5knFX/src/dual.jl:24
  ...